# Module 12.3: Self-Service Tenant Onboarding

**Goal:** Automate SaaS customer onboarding using multi-tenant architecture and billing integration.

**Duration:** 38 minutes

**Level:** 3 (Advanced)

## Setup and Imports

First, let's import the module and configure logging.

In [ ]:
import sys
import os
import logging
from datetime import datetime
import json

# Add parent directory to path for imports
sys.path.insert(0, os.path.dirname(os.path.dirname(os.path.abspath('__file__'))))

# Import module functions
from src.l3_m12_self_service_tenant_onboarding import *
import config

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

print("✓ Module loaded successfully")
print(f"Timestamp: {datetime.utcnow().isoformat()}")

## 1. Core Problem Statement

**The Challenge:**
> "Every new customer requires manual setup: creating tenant records, configuring namespaces, sending credentials, and conducting setup calls—consuming 10-15 hours weekly."

**The Solution:**
Automate the entire onboarding flow, enabling customers to become productive within **5 minutes** while you sleep.

**Key Benefits:**
- Reduce manual work from 15 hours/week to near-zero
- Improve customer time-to-value (5 minutes vs. 2-5 days)
- Scale to 100+ daily signups without additional headcount
- Increase activation rates through guided setup wizard

## 2. Architecture Overview

The onboarding journey follows **four stages**:

### Stage 1: Signup & Payment Capture (Synchronous)
- User submits email, company name, password, and plan selection
- Stripe Checkout collects payment information
- Returns immediately with status `pending_payment`

### Stage 2: Automated Provisioning (Background via Celery)
- Creates Pinecone namespace
- Creates database tables
- Creates Stripe subscription
- Generates API keys
- Status transitions to `active`

### Stage 3: Welcome & Activation (Email + In-App)
- Sends welcome emails with login links and docs
- Interactive wizard guides users through:
  - Uploading first document
  - Running first query
  
### Stage 4: Activation Monitoring (Analytics)
- Tracks conversion metrics
- Identifies where users drop off
- Typical funnel: 30% after payment, 20% at login, 50% reaching first successful query within 24 hours

## 3. Step 1: Public Signup Endpoint

Creates skeleton tenant record **before** payment collection. Generates unique `tenant_id`, creates admin user with hashed password, and returns Stripe Checkout URL. Status remains `pending_payment` until webhook confirmation.

In [ ]:
# Create skeleton tenant
tenant = create_skeleton_tenant(
    email="demo@startup.com",
    company_name="Startup Inc",
    password="SecurePass123!",
    plan="pro"
)

print(f"Tenant ID: {tenant['tenant_id']}")
print(f"Status: {tenant['status']}")
print(f"Plan: {tenant['plan']}")

# Generate checkout URL (will be mock if Stripe not configured)
checkout_url = generate_stripe_checkout_url(tenant['tenant_id'], tenant['plan'])
print(f"Checkout URL: {checkout_url[:60]}...")

# Expected:
# Tenant ID: <32-char hex string>
# Status: pending_payment
# Plan: pro

## 4. Step 2: Stripe Webhook Handler

Listens for `checkout.session.completed` event, updates tenant status to `provisioning`, stores Stripe customer/subscription IDs, and triggers background provisioning task via Celery.

In [ ]:
# Simulate webhook event (in-memory tenant store)
tenant_store = {tenant['tenant_id']: tenant}

webhook_data = {
    'object': {
        'metadata': {'tenant_id': tenant['tenant_id']},
        'customer': 'cus_mock123',
        'subscription': 'sub_mock456',
    }
}

# Handle webhook
updated_tenant = handle_checkout_completed(webhook_data, tenant_store)

print(f"Status updated: {updated_tenant['status']}")
print(f"Stripe Customer ID: {updated_tenant['stripe_customer_id']}")
print(f"Stripe Subscription ID: {updated_tenant['stripe_subscription_id']}")

# Expected:
# Status updated: provisioning
# Stripe Customer ID: cus_mock123
# Stripe Subscription ID: sub_mock456

## 5. Step 3: Celery Background Provisioning Job

Executes asynchronously:
1. Creates Pinecone namespace
2. Provisions database schema
3. Creates Stripe subscription
4. Generates API keys
5. Loads sample data
6. Updates tenant status to `active`

Includes retry logic and error handling for resilience.

In [ ]:
# Load sample documents
import os
data_path = os.path.join(os.path.dirname(os.path.dirname(os.path.abspath('__file__'))), 'example_data.json')
with open(data_path, 'r') as f:
    data = json.load(f)
    sample_docs = data.get('sample_documents', [])

# Provision tenant (will skip external API calls if clients not configured)
provisioned = provision_tenant(
    tenant['tenant_id'],
    tenant_store,
    pinecone_client=config.get_pinecone_client(),
    stripe_client=config.get_stripe_client(),
    sample_docs=sample_docs
)

print(f"Provisioning Status: {provisioned['status']}")
print(f"Pinecone Namespace: {provisioned.get('pinecone_namespace', 'N/A')}")
print(f"API Key: {provisioned.get('api_key', 'N/A')[:40]}...")
print(f"Sample Docs Loaded: {provisioned.get('sample_docs_loaded', 0)}")

# Expected:
# Provisioning Status: active
# Pinecone Namespace: tenant_<tenant_id>
# API Key: <JWT token>
# ⚠️ Skipping API calls if services not configured

## 6. Step 4: Interactive Setup Wizard

Provides guided interface for:
- Document upload
- Query execution
- Integration exploration

Reduces first-time user friction and improves activation rates.

In [ ]:
# Get setup wizard steps
wizard_steps = get_setup_wizard_steps()

print("Setup Wizard Steps:")
for step in wizard_steps:
    print(f"\nStep {step['step']}: {step['title']}")
    print(f"  → {step['description']}")
    print(f"  Action: {step['action']}")

# Expected:
# Step 1: Upload First Document
# Step 2: Run First Query
# Step 3: Explore Integrations

## 7. Step 5: Activation Metrics Dashboard

Tracks signup-to-activation pipeline:
- Email opened rate
- Login completion
- First document upload
- First successful query

Identifies bottlenecks for continuous optimization.

In [ ]:
# Simulate activation events
events = [
    track_activation_event(tenant['tenant_id'], 'signup_completed'),
    track_activation_event(tenant['tenant_id'], 'payment_confirmed'),
    track_activation_event(tenant['tenant_id'], 'first_login'),
    track_activation_event(tenant['tenant_id'], 'first_document_upload'),
    track_activation_event(tenant['tenant_id'], 'first_query_executed'),
]

# Calculate metrics
metrics = calculate_activation_metrics(events)

print("Activation Metrics:")
print(f"  Total Signups: {metrics['total_signups']}")
print(f"  Payment Confirmed: {metrics['payment_confirmed']}")
print(f"  First Login: {metrics['first_login']}")
print(f"  First Query: {metrics['first_query_executed']}")
print(f"\n  Login Rate: {metrics['login_rate']}%")
print(f"  Activation Rate: {metrics['activation_rate']}%")

# Expected:
# Activation Rate: 100% (single user completed full funnel)
# In production: track across all users to find drop-off points

## 8. Production Considerations

### Scaling to 100+ Daily Signups

**Required optimizations:**
- Rate limiting on signup endpoint
- Redis caching for tenant lookups
- Monitor Celery queue depth; scale workers horizontally
- Implement idempotent provisioning tasks

### Cost Breakdown at Scale

- **Stripe processing:** 2.9% + $0.30 per transaction
- **Pinecone:** $0.04-0.40 per 1M vectors (pod-dependent)
- **Celery/Redis:** Negligible with proper resource pooling
- **Email delivery:** $0.0001-0.001 per message via SendGrid

### Monitoring Metrics

Track:
- Signup funnel completion rates
- Provisioning job success rates
- Mean time-to-activation
- Webhook delivery latency

**Alert when:**
- Task queue backlog >100 jobs
- Provisioning success rate <95%
- Webhook latency >30 seconds

### Security & Compliance

- ✓ Webhook signature verification prevents unauthorized provisioning
- ✓ API keys use tenant-scoped claims in JWT
- ✓ Sample data loads use temporary credentials with restricted permissions
- ✓ Rate limiting to prevent abuse of signup endpoint

**Compliance:**
- **GDPR:** Store consent for marketing emails; implement unsubscribe mechanisms
- **SOC2:** Audit webhook processing; maintain event logs for 90 days
- **PCI:** Never store raw card data; rely on Stripe tokenization

## 9. When NOT to Use Self-Service

Self-service onboarding breaks in three scenarios:

### ❌ Scenario 1: Enterprise Sales
Companies requiring custom contracts, extensive security reviews, and dedicated onboarding shouldn't face automated signup. **Sales-assisted flow is appropriate.**

### ❌ Scenario 2: Complex Product
Products requiring expert configuration or domain knowledge benefit from **guided implementation services** rather than self-service wizards.

### ❌ Scenario 3: Long Sales Cycles
B2B products with multiple stakeholder approvals need **human touch points**, not automated provisioning.

**Rule of thumb:** If customer LTV >$50K/year, use sales-assisted onboarding.

## 10. Common Failure Modes

### 🔴 Failure 1: Provisioning Job Crashes
**Symptom:** Tenant remains in "provisioning" limbo indefinitely. User sees loading screen, gives up.

**Solution:** Implement timeout logic; transition to "failed" status with retry button after 5 minutes.

### 🔴 Failure 2: Wizard Complexity
**Symptom:** Users drop off at document upload step.

**Solution:** A/B test simpler 2-step wizard (upload, query) vs. detailed configuration. Track dropout rates per step.

### 🔴 Failure 3: Sample Data Loading Fails
**Symptom:** New tenant appears broken on first login.

**Solution:** Pre-generate sample data during provisioning; include error message with fallback to empty account.

### 🔴 Failure 4: Inaccurate Activation Tracking
**Symptom:** Metrics don't reflect actual user behavior due to client-side tracking failures.

**Solution:** Server-side event logging; validate client events against backend logs.

### 🔴 Failure 5: Welcome Email to Spam
**Symptom:** Customers never receive login credentials.

**Solution:** Use authenticated domains (SPF/DKIM/DMARC); include preview text; avoid spam trigger words.

In [ ]:
# Demo: Failure Mode #1 - Provisioning Timeout Detection
from datetime import timedelta

# Simulate old provisioning tenant
old_tenant = {
    'tenant_id': 'test_123',
    'status': TenantStatus.PROVISIONING.value,
    'updated_at': (datetime.utcnow() - timedelta(minutes=10)).isoformat()
}

# Check for timeout
timed_out = check_provisioning_timeout(old_tenant, timeout_seconds=300)

print(f"Provisioning timed out: {timed_out}")
if timed_out:
    print("  → Should transition to 'failed' status")
    print("  → Show retry button to user")

# Expected:
# Provisioning timed out: True
# (Tenant has been provisioning for >5 minutes)

## 11. Alternative Approaches

### Sales-Assisted (High-Touch)
**How it works:**
- Customers request demo
- Sales rep provisions tenant manually
- Conducts onboarding call

**Pros:** Better conversion for enterprise deals  
**Cons:** Slower (2-5 days)  
**Use when:** Customer LTV >$50K/year

### Waitlist with Manual Approval
**How it works:**
- Control growth; vet customers
- Review applications daily
- Approve/deny within 24 hours

**Pros:** Prevents abuse during beta  
**Cons:** Slower than pure self-service  
**Use when:** Limited capacity or beta phase

### Hybrid Model (Recommended)
**How it works:**
- Self-service for SMB (Starter/Pro plans)
- Sales-assisted for Enterprise tier
- Monitor activation rates
- Escalate at-risk customers to support team for intervention calls

**Pros:** Best of both worlds  
**Cons:** More complex to implement  
**Use when:** Serving multiple market segments

## 12. Decision Framework

Use this decision card to determine if self-service onboarding is right for your product:

### ✅ Use Self-Service When:

| Criterion | Threshold |
|-----------|-----------|
| **Target market** | SMB/mid-market (not enterprise) |
| **Product simplicity** | Intuitive core workflows |
| **Sales cycle** | <1 week |
| **Unit economics** | Sustainable at $100+ MRR |

### ❌ Avoid When:

| Criterion | Reason |
|-----------|--------|
| **Enterprise-first strategy** | Custom contracts required |
| **Significant implementation** | Expert configuration needed |
| **Complex integrations** | Prerequisites for setup |
| **Compliance/security reviews** | Manual vetting necessary |

### Decision Tree

```
Is target market SMB/mid-market?
├─ NO → Sales-assisted
└─ YES → Is product self-explanatory?
    ├─ NO → Hybrid (wizard + support)
    └─ YES → Is sales cycle <1 week?
        ├─ NO → Waitlist + approval
        └─ YES → ✅ Self-Service
```

## 13. Key Takeaway

> **"Self-service onboarding doesn't mean abandoned customers—it means _automated_ onboarding with _monitored_ activation. Build the automation pipeline, track where customers stick, and intervene when they struggle."**

### Summary

This module covered:

1. ✓ **Problem:** Manual onboarding consumes 10-15 hours weekly
2. ✓ **Solution:** 4-stage automated pipeline (signup → provisioning → activation → monitoring)
3. ✓ **Implementation:** Step-by-step guide with Stripe, Pinecone, Celery, SendGrid
4. ✓ **Production:** Scaling considerations, cost breakdown, monitoring metrics
5. ✓ **Failure Modes:** 5 common failures and their fixes
6. ✓ **Decision Framework:** When to use self-service vs. sales-assisted

### Next Steps

- **Module 12.4:** Usage-based billing and metering
- **Module 13.1:** Multi-region deployment strategies
- **Module 13.2:** Disaster recovery and backup automation

### Practathon Challenges

- **Easy (60-90 min):** Build signup endpoint + basic welcome email
- **Medium (2-3 hours):** Add Celery provisioning job + Pinecone namespace creation
- **Hard (5-6 hours):** Complete system with setup wizard, activation tracking dashboard, and failure recovery logic